<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [1]:
import numpy as np
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10, sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector

### create & preprocess dataset

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        # create X
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        # create y
        out_pattern = sum(in_pattern)
        # append
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

X, y = random_sum_pairs(10, 4, 20)
print(X[:2])
print(y[:2])

[[3, 5, 14, 12], [8, 15, 19, 16]]
[34, 58]


In [3]:
# ceil --> function always rounds a number up to the next largest integer.

def pairs_to_string(X, y, n_numbers, largest):
    # calmax len
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    # convert to str and pading X
    Xstr = []
    for p in X:
        strp = '+'.join([str(n) for n in p])
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    
    # convert to str and pading y
    ystr = []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

Xstr, ystr = pairs_to_string(X, y, 4, 20)
print(Xstr[:2])
print(ystr[:2])

['  3+5+14+12', ' 8+15+19+16']
['34', '58']


In [4]:
alphabets = set(''.join(Xstr))

def integer_encode(X, y, alphabets):
    # creat incoder
    char_to_int = dict((c, i) for i, c in enumerate(alphabets))
    # print(char_to_int)
    
    #incod X
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    # incod y
    yenc = []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return Xenc, yenc

integer_encode(Xstr, ystr, alphabets)[0][:2]

[[4, 4, 0, 3, 6, 3, 9, 5, 3, 9, 2], [4, 7, 3, 9, 6, 3, 9, 11, 3, 9, 8]]

In [5]:
def one_hot_encode(X, y, max_int):
    Xenc = []
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
        
    yenc = []
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [6]:
alphabets = set(''.join(Xstr))

def integer_decode_IncoderOnehot(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets))
    strings = []
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

integer_decode_IncoderOnehot([[0,0,1,0,0,0,0,0,0,0,0]], alphabets)

'2'

In [7]:
# call all fun to create data
def generate_data(n_samples, n_numbers, largest, alphabets):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = pairs_to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabets)
    X, y = one_hot_encode(X, y, len(alphabets))
    X, y = array(X), array(y)
    return X, y


seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = set(''.join(Xstr))
n_chars = len(alphabets)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

X, y = generate_data(n_samples, n_numbers, largest, alphabets)
print(X.shape)
print(y.shape)

(1000, 8, 12)
(1000, 2, 12)


### create & fit model

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 64)             49408     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            780       
 ibuted)                                                         
                                                                 
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X, y, epochs=2, batch_size=50)

Epoch 1/2
20/20 [==============================] - 0s 14ms/step - loss: 0.1016 - accuracy: 0.9935
Epoch 2/2
20/20 [==============================] - 0s 11ms/step - loss: 0.1011 - accuracy: 0.9925


### test model

In [13]:
# generat new data
X, y = generate_data(n_samples, n_numbers, largest, alphabets)

# #predict
# result = model.predict(X, batch_size=50)
# # print(integer_decode(result[0], alphabets))

# # decod
# expected = [integer_decode(y1, alphabets) for y1 in y]
# predicted = [integer_decode(y1, alphabets) for y1 in result]

# print('loss : ' + str(np.abs(np.array(expected, dtype='int') - np.array(predicted, dtype='int')).sum() / len(expected)))

# test 
model.evaluate(X,y)

32/32 [==============================] - 1s 11ms/step - loss: 0.2964 - accuracy: 0.9060


[0.2963758111000061, 0.906000018119812]

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Regression</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>